In [ ]:
import heapq
import random
from collections import deque
from typing import Dict, List, Set

**380. Insert Delete GetRandom O(1)**


In [5]:
class RandomizedSet:  # 68% time, 8% memory
    """
    Super clever solution by elliot charney
    """

    def __init__(self):
        # store = {Value: Index of Value in self.array, ...}
        self.store = {}
        # array = [Value, ...]
        self.array = []

    def insert(self, val: int) -> bool:
        if val in self.store:
            return False

        # store the value and its index
        self.store[val] = len(self.array)
        self.array.append(val)
        return True

    def remove(self, val: int) -> bool:
        if val not in self.store:
            return False

        # move the entry that is last in the array to the index of the value being popped
        # this is faster than shifting the whole array on the right side of the index to the left by 1 O(n)
        index_to_replace = self.store[val]
        self.array[index_to_replace] = self.array[-1]

        # update the index of the moved value
        self.store[self.array[-1]] = index_to_replace
        self.array.pop()  # get rid of the duplicate value

        del self.store[val]

        return True

    def getRandom(self) -> int:
        return random.choice(self.array)

**432. All O(1) Data Structure**


In [6]:
# 51% time, 15% memory


class Node:
    def __init__(self, count: int, prev=None, next=None) -> None:
        self.keys: Set[str] = set()
        self.count = count
        self.prev: "Node" = prev  # type: ignore
        self.next: "Node" = next  # type: ignore

    def __repr__(self) -> str:
        return (f"Node[{self.count}: {self.keys}]" if self.count else "End") + (
            f" <-> {repr(self.next)}" if self.next is not None else ""
        )


class AllOne:
    """
    Doubly linked lists will be used to track Max and Min Counts
    Nodes will just be sets with metadata (frequency of each of the words in the set)

    - So words will be grouped by count into Nodes
    - Words will point to their current Node
    - There will be a head and tail node pointing to the current largest and current smallest count Nodes
    """

    def __init__(self):
        """
        Head will point to the max (so it is the last item in the list)
        Tail will point to the min (so it is the first item in the list)
        The Zeroes are just to flag an empty list
        """
        self.head, self.tail = Node(0), Node(0)
        self.head.prev, self.tail.next = self.tail, self.head

        # {word : Node that the word currently belongs to}
        self.word_to_node: Dict[str, Node] = {}

    def delete(self, node: Node):
        node.prev.next = node.next
        node.next.prev = node.prev

    def inc(self, key: str) -> None:
        """
        2 Situations
        - New node
        - Not new node
        If the node at current value + 1 doesn't exist, make it and insert it
        """
        # node is either the current word's node or the tail node if the word is new
        node = self.word_to_node.get(key, self.tail)
        newCount = node.count + 1

        if node.next.count != newCount:
            oldNextNode = node.next
            # make new node
            nextNode = Node(newCount, prev=node, next=oldNextNode)
            # insert new node
            node.next = nextNode
            oldNextNode.prev = nextNode
        else:
            nextNode = node.next

        # move key from current to next node
        nextNode.keys.add(key)
        node.keys.discard(
            key
        )  # doesn't raise an exception when we try to remove from self.tail.keys

        if node.count > 0 and not node.keys:
            # if the node is not the tail and the node is empty, delete it
            self.delete(node)

        # update pointer
        self.word_to_node[key] = nextNode

    def dec(self, key: str) -> None:
        # node is the current word's node
        node = self.word_to_node[key]
        newCount = node.count - 1

        if node.prev.count != newCount:
            oldPrevNode = node.prev
            # make new node
            prevNode = Node(newCount, prev=oldPrevNode, next=node)
            # insert new node
            node.prev = prevNode
            oldPrevNode.next = prevNode
        else:
            prevNode = node.prev

        # move key from current to prev node
        if newCount:
            prevNode.keys.add(key)
        # can use remove here since it is guaranteed to have it
        node.keys.remove(key)

        if not node.keys:
            # if the node is empty, delete it
            self.delete(node)

        # update pointer
        if newCount:
            self.word_to_node[key] = prevNode
        else:
            del self.word_to_node[key]

    def getMaxKey(self) -> str:
        if self.head.prev.count == 0:
            return ""
        return next(iter(self.head.prev.keys))

    def getMinKey(self) -> str:
        if self.tail.next.count == 0:
            return ""
        return next(iter(self.tail.next.keys))


# Your AllOne object will be instantiated and called as such:
obj = AllOne()
obj.inc("hello")
obj.inc("hello")
print(obj.getMaxKey())
print(obj.getMinKey())
obj.inc("leet")
print(obj.getMaxKey())
print(obj.getMinKey())

print(obj.tail)

hello
hello
hello
leet
End <-> Node[1: {'leet'}] <-> Node[2: {'hello'}] <-> End


**381. Insert Delete GetRandom O(1) - Duplicates allowed**


In [25]:
# 71% time, 17% memory
class RandomizedCollection:
    """
    I'm thinking the exact same thing as the version with no duplicates
    But instead of a hashmap where the values are an index, the values need to be a list of indices
    Then you pop and append (both O(1)) to those lists on remove and insert, repectively
    """

    def __init__(self):
        # {val: {index1, index2, ...}}
        self.store: Dict[int, Set[int]] = {}
        # the underlying array of raw vals
        self.data = []

    def insert(self, val: int) -> bool:
        flag = False
        if val not in self.store:
            flag = True
            self.store[val] = set()

        self.store[val].add(len(self.data))
        self.data.append(val)

        # True if new insert, False otherwise
        return flag

    def remove(self, val: int) -> bool:
        """
        I want to move the value at the end of data
        """
        if val not in self.store:
            return False

        indexToReplace = self.store[val].pop()
        valueReplacingIt = self.data.pop()
        if len(self.store[val]) == 0:
            del self.store[val]

        if indexToReplace != len(self.data):
            self.store[valueReplacingIt].remove(len(self.data))
            self.store[valueReplacingIt].add(indexToReplace)
            self.data[indexToReplace] = valueReplacingIt

        return True

    def getRandom(self) -> int:
        return random.choice(self.data)


# Your RandomizedCollection object will be instantiated and called as such:
obj = RandomizedCollection()
dct = {"insert": obj.insert, "remove": obj.remove, "getRandom": obj.getRandom}

cmds, vals = (
    [
        "RandomizedCollection",
        "insert",
        "insert",
        "insert",
        "insert",
        "insert",
        "insert",
        "remove",
        "remove",
        "remove",
        "remove",
        "remove",
        "insert",
        "remove",
        "remove",
        "getRandom",
        "getRandom",
        "getRandom",
        "getRandom",
        "getRandom",
        "getRandom",
        "getRandom",
        "getRandom",
        "getRandom",
        "getRandom",
    ],
    [
        [],
        [10],
        [10],
        [20],
        [20],
        [30],
        [30],
        [10],
        [20],
        [20],
        [10],
        [30],
        [40],
        [30],
        [30],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
    ],
)
for cmd, val in zip(cmds, vals):
    if cmd in dct:
        print((cmd, *val))
        dct[cmd](*val)  # type: ignore
        print("\t", (cmd, *val), obj.store, obj.data)

('insert', 10)
	 ('insert', 10) {10: {0}} [10]
('insert', 10)
	 ('insert', 10) {10: {0, 1}} [10, 10]
('insert', 20)
	 ('insert', 20) {10: {0, 1}, 20: {2}} [10, 10, 20]
('insert', 20)
	 ('insert', 20) {10: {0, 1}, 20: {2, 3}} [10, 10, 20, 20]
('insert', 30)
	 ('insert', 30) {10: {0, 1}, 20: {2, 3}, 30: {4}} [10, 10, 20, 20, 30]
('insert', 30)
	 ('insert', 30) {10: {0, 1}, 20: {2, 3}, 30: {4, 5}} [10, 10, 20, 20, 30, 30]
('remove', 10)
	 ('remove', 10) {10: {1}, 20: {2, 3}, 30: {0, 4}} [30, 10, 20, 20, 30]
('remove', 20)
	 ('remove', 20) {10: {1}, 20: {3}, 30: {0, 2}} [30, 10, 30, 20]
('remove', 20)
	 ('remove', 20) {10: {1}, 30: {0, 2}} [30, 10, 30]
('remove', 10)
	 ('remove', 10) {30: {0, 1}} [30, 30]
('remove', 30)
	 ('remove', 30) {30: {0}} [30]
('insert', 40)
	 ('insert', 40) {30: {0}, 40: {1}} [30, 40]
('remove', 30)
	 ('remove', 30) {40: {0}} [40]
('remove', 30)
	 ('remove', 30) {40: {0}} [40]
('getRandom',)
	 ('getRandom',) {40: {0}} [40]
('getRandom',)
	 ('getRandom',) {40: {0}}

**2349. Design a Number Container System**


In [ ]:
# 76% time, 99% memory
class NumberContainers:
    """
    So it turns out that my original heap solution is way faster... LOLOLOL
    even though it is O(nlogn) on every `find` the `change` is O(log(n)) instead of O(n)
    """

    def __init__(self):
        self.numToIndices: Dict[int, List[int]] = {}
        self.indexToNum: Dict[int, int] = {}

    def change(self, index: int, number: int) -> None:
        self.indexToNum[index] = number
        if number in self.numToIndices:
            heapq.heappush(self.numToIndices[number], index)
        else:
            self.numToIndices[number] = [index]

    def find(self, number: int) -> int:
        if number not in self.numToIndices:
            return -1
        minHeap = self.numToIndices[number]
        while minHeap and self.indexToNum[minHeap[0]] != number:
            # since we aren't pruning the heap in `change` we need to check for dirty indices
            heapq.heappop(minHeap)
        if minHeap:
            return minHeap[0]
        return -1


# Your NumberContainers object will be instantiated and called as such:
# obj = NumberContainers()
# obj.change(index,number)
# param_2 = obj.find(number)

**1352. Product of the Last K Numbers**


In [ ]:
# 21% time, 37% memory
class ProductOfNumbers:
    """
    If you keep track of the products for all contiguous arrays from index 0 to n
    Where n is the aount of integers added up to that point
    Then you can just grab the one at `n` and divide it by the one at `n - k - 1`
    """

    def __init__(self):
        self.data = [0]

    def add(self, num: int) -> None:
        if num == 0:
            self.data = [0] * (len(self.data) + 1)
        else:
            self.data.append(max(self.data[-1], 1) * num)

    def getProduct(self, k: int) -> int:
        if self.data[-k] == 0:
            return 0
        return self.data[-1] // max(self.data[-k - 1], 1)


# Your ProductOfNumbers object will be instantiated and called as such:
obj = ProductOfNumbers()
obj.add(9)
obj.add(2)
obj.add(3)
obj.add(4)
print(obj.getProduct(1))
print(obj.getProduct(2))
print(obj.getProduct(3))
print(obj.getProduct(4))

4
12
24
216
